# Assignment 1, Qilin Zhou, 01/11/2024

## Question 1

### Data Preparation

In [42]:
import sys
import sklearn
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
import urllib.request

In [43]:
datapath = os.path.join("datasets", "lifesat", "")
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
os.makedirs(datapath, exist_ok=True)
for filename in ("oecd_bli_2015.csv", "gdp_per_capita.csv"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "datasets/lifesat/" + filename
    urllib.request.urlretrieve(url, datapath + filename)

In [44]:
oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
gdp_per_capita = pd.read_csv(datapath + "gdp_per_capita.csv",thousands=',',delimiter='\t',
                             encoding='latin1', na_values="n/a")

In [45]:
oecd_bli = oecd_bli[oecd_bli.Indicator == "Life satisfaction"][oecd_bli.INEQUALITY == "TOT"][["Country", "Value"]]

/var/folders/97/r7d175650dg4fx0f_n5km01w0000gn/T/ipykernel_20329/2958508254.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  oecd_bli = oecd_bli[oecd_bli.Indicator == "Life satisfaction"][oecd_bli.INEQUALITY == "TOT"][["Country", "Value"]]


In [46]:
oecd_bli = oecd_bli[oecd_bli.Country != "OECD - Total"]
oecd_bli.set_index("Country", inplace=True)

In [47]:
gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
gdp_per_capita = gdp_per_capita[["Country", "GDP per capita"]]
gdp_per_capita.set_index("Country", inplace=True)

In [49]:
full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita, left_index=True, right_index=True)
full_country_stats.rename(columns={"Value": "Life Satisfaction"}, inplace=True)

### Scikit-learn Models

In [51]:
X = np.c_[full_country_stats["GDP per capita"]]
y = np.c_[full_country_stats["Life Satisfaction"]] 

In [52]:
X_new = [[22587]]  # Cyprus' GDP per capita

#### Model 1: K-Nearest Neighbors Regression

In [53]:
import sklearn.neighbors

kn_model = sklearn.neighbors.KNeighborsRegressor(
    n_neighbors=4)

kn_model.fit(X, y)

KNeighborsRegressor(n_neighbors=4)

In [54]:
print(kn_model.predict(X_new))

[[5.525]]


#### Model 2: Decision Tree Regressor

In [55]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor()

tree_model.fit(X, y)

DecisionTreeRegressor()

In [56]:
print(tree_model.predict(X_new))

[5.7]


#### Model 3: Random Forest Regressor

In [57]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()

rf_model.fit(X, y)

/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [58]:
print(rf_model.predict(X_new))

[5.742]


## Question 2

### Load Data

In [59]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [60]:
fetch_housing_data()

In [61]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [62]:
housing = load_housing_data()

### Split Data into Test and Train Sets

In [63]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [64]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [65]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [68]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

### Data Preprocessing

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin

col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [79]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

housing_num = housing.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared.shape

(16512, 16)

### Select, Train, and Fine-Tune a Model

In [90]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

#### GridSearchCV

In [96]:
param_grid = [{"kernel": ["linear"], "C": [1, 10, 100]},
              {"kernel": ["rbf"], "C":[1, 10, 100], "gamma":[1, 10]},
              ]

svm_reg = SVR()

grid_search = GridSearchCV(SVR(), param_grid, cv=5,
                           scoring="neg_mean_squared_error",
                           return_train_score=True, n_jobs=12)

grid_search.fit(housing_prepared, housing_labels)
grid_search.best_params_

{'C': 100, 'kernel': 'linear'}

In [104]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

112571.06378605746 {'C': 1, 'kernel': 'linear'}
84649.6069847477 {'C': 10, 'kernel': 'linear'}
71635.55362813479 {'C': 100, 'kernel': 'linear'}
118898.89058474178 {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
118939.18538817228 {'C': 1, 'gamma': 10, 'kernel': 'rbf'}
118591.6498917307 {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
118933.10516441689 {'C': 10, 'gamma': 10, 'kernel': 'rbf'}
115840.14747601148 {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
118893.07619028944 {'C': 100, 'gamma': 10, 'kernel': 'rbf'}


#### RandomizedSearchCV

In [107]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {"kernel": ["linear"], 'C': randint(low=1, high=100)}

rnd_search = RandomizedSearchCV(SVR(), param_distributions=param_distribs,
                                n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=12)

rnd_search.fit(housing_prepared, housing_labels)
rnd_search.best_params_

{'C': 93, 'kernel': 'linear'}

In [108]:
rnd_cvres = rnd_search.cv_results_
for mean_score, params in zip(rnd_cvres["mean_test_score"], rnd_cvres["params"]):
    print(np.sqrt(-mean_score), params)

73127.97176777973 {'C': 52, 'kernel': 'linear'}
71746.03296245693 {'C': 93, 'kernel': 'linear'}
80562.06081651486 {'C': 15, 'kernel': 'linear'}
72274.72993642453 {'C': 72, 'kernel': 'linear'}
72672.31748065217 {'C': 61, 'kernel': 'linear'}
77654.58448305773 {'C': 21, 'kernel': 'linear'}
71969.10641598784 {'C': 83, 'kernel': 'linear'}
71863.55753820174 {'C': 87, 'kernel': 'linear'}
72176.78380457204 {'C': 75, 'kernel': 'linear'}
72176.78380457204 {'C': 75, 'kernel': 'linear'}
71846.16459544172 {'C': 88, 'kernel': 'linear'}
76720.96107554667 {'C': 24, 'kernel': 'linear'}
102376.3300964536 {'C': 3, 'kernel': 'linear'}
77328.07564454456 {'C': 22, 'kernel': 'linear'}
73065.60480106607 {'C': 53, 'kernel': 'linear'}
107140.2596216261 {'C': 2, 'kernel': 'linear'}
71846.16459544172 {'C': 88, 'kernel': 'linear'}
75457.08962934253 {'C': 30, 'kernel': 'linear'}
74329.66822509131 {'C': 38, 'kernel': 'linear'}
107140.2596216261 {'C': 2, 'kernel': 'linear'}


* By comparing the RMSE scores in the grid search best estimators and randomized search best estimators (71635.55 < 71746.03), we obtain the best solution by setting the kernel as linear with C hyperparameter as 100.

### Evaluate on the Test Set

In [116]:
from sklearn.metrics import mean_squared_error
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 
final_rmse

69499.58221771519

In [111]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([67011.23772994, 71901.86280463])

In [115]:
from sklearn.metrics import mean_absolute_error
final_mae = mean_absolute_error(y_test, final_predictions)
final_mae

48262.189839076214

In [113]:
y_test.describe()

count      4128.000000
mean     206257.795058
std      114176.653346
min       14999.000000
25%      118900.000000
50%      181300.000000
75%      268850.000000
max      500001.000000
Name: median_house_value, dtype: float64

* The test RMSE is 69499.58 within the 95 CI confidence interval. Since the median is around 181300, an RMSE of 69499.58 has a relative error of around 38%, which is quite high. After accounting for the potential outlier such as the minimum median housing value which is 14999, an MAE of 48262.19 has a relative error of around 27%. The model may need further hyperparameter tuning and attribute revision, depending on the specific error bound required.

## Question 3

In [142]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
# Recall that housing = strat_train_set.drop("median_house_value", axis=1)
# housing_labels = strat_train_set["median_house_value"].copy()

class ImportantFeaturesSelector(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.1):
        self.threshold = threshold
        self.important_indices_ = None

    def fit(self, X, y):
        corr = np.abs(np.corrcoef(X, y, rowvar=False)[:-1, -1])
        self.important_indices_ = np.where(corr > self.threshold)[0]
        return self
    
    def transform(self, X):
        return X[:, self.important_indices_] 

In [143]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

housing_num = housing.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('important_features', ImportantFeaturesSelector(threshold=0.1)),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

new_housing_prepared = full_pipeline.fit_transform(housing, housing_labels)

In [144]:
new_housing_prepared.shape

(16512, 11)